<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 8 Assignment: Feature Engineering**

**Student Name: Your Name**

# Assignment Instructions

This assignment is similar to assignment 5, except that you must use feature engineering to solve it.  I provide you with a dataset that contains dimensions and the quality of items of specific shapes.  With the values of 'height', 'width', 'depth'. 'shape', and 'quality' you should try to predict the cost of these items.  You should be able to match very close to solution file, if you feature engineer correctly.  To get full credit your average cost should not be more than 50 off from the solution.  The autocorrector will let you know if you are in this range.

You can find all of the needed CSV files here:

* [Shapes - Training](https://data.heatonresearch.com/data/t81-558/datasets/shapes-train.csv)
* [Shapes - Submit](https://data.heatonresearch.com/data/t81-558/datasets/shapes-test.csv)

Use the training file to train your neural network and submit results for for the data contained in the test/submit file.

In [22]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [23]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #8 Sample Code

The following code provides a starting point for this assignment.

In [24]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy import stats 
from tensorflow.keras import regularizers
import math

# This is your student key that I emailed to you at the beginnning of the semester.
key = "l15XevJfTL81Eylq9Eaw17IR3RqfdfHC8LXMvy6a"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/My Drive/Colab Notebooks/assignments/assignment_victormacia_class8.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class8.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class8.ipynb'  # Mac/Linux





In [25]:
# Begin assignment

df_train = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/shapes-train.csv")
df_submit = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/shapes-test.csv")

df_train2 = df_train 
df_submit2 = df_submit

#df_train['cost'] = (df_train['cost']-df_train['cost'].mean())/df_train.cost.std()

In [26]:
#df_train['cost'] = np.log(df_train['cost']+np.ones(len(df_train.cost)))

In [27]:

for i in df_train.columns.difference(['shape','id']):
    df_train[i] = df_train[i].astype(float)


df_train




,id,height,width,depth,shape,quality,cost
0,1,40.0,53.0,89.0,cylinder,89.0,200.49
1,2,85.0,35.0,51.0,box,37.0,1175.71
2,3,12.0,19.0,61.0,box,23.0,131.72
3,4,29.0,37.0,20.0,box,94.0,15.83
4,5,87.0,40.0,22.0,cylinder,54.0,340.21
...,...,...,...,...,...,...,...
9995,9996,96.0,80.0,68.0,box,39.0,3918.36
9996,9997,55.0,36.0,93.0,ellipsoid,96.0,35.57
9997,9998,20.0,69.0,32.0,box,40.0,325.90
9998,9999,52.0,95.0,99.0,box,92.0,481.23


In [28]:
# Calculating volumes

# volume cylinder
#
# cylinder_vol = area base x height = pi x (width/2)^2 x height
# box_vol = heigth*width*depth
# elip_vol = width x height x length x pi / 6

df_train['volume'] = len(df_train['shape'])*[0]
df_train['base_area'] = len(df_train['shape'])*[0]

def volume(df_train):
      for i in df_train[df_train['shape'] == 'cylinder'].index:
              df_train.volume[i] = math.pi*(df_train.width[i]/2.0)**2*df_train.height[i]
      for i in df_train[df_train['shape'] == 'box'].index:
              df_train.volume[i] = df_train.width[i]*df_train.height[i]*df_train.depth[i]
      for i in df_train[df_train['shape'] == 'ellipsoid'].index:
              df_train.volume[i] = df_train.width[i]*df_train.height[i]*df_train.depth[i]*(math.pi/6.0)
      return df_train

def area(df_train):
      for i in df_train[df_train['shape'] == 'cylinder'].index:
              df_train.base_area[i] = math.pi*(df_train.width[i]/2.0)**2
      for i in df_train[df_train['shape'] == 'box'].index:
              df_train.base_area[i] = df_train.width[i]*df_train.depth[i]
      for i in df_train[df_train['shape'] == 'ellipsoid'].index:
              df_train.base_area[i] = df_train.width[i]*df_train.depth[i]*(math.pi/6.0)
      return df_train

df_train = volume(df_train)
df_train = area(df_train)
df_train


# submit dataset

df_submit2['volume'] = len(df_submit['shape'])*[0.0]
df_submit2['base_area'] = len(df_submit['shape'])*[0.0]

def volume(df_submit2):
      for i in df_submit2[df_submit2['shape'] == 'cylinder'].index:
              df_submit2.volume[i] = float(math.pi)*(float(df_submit2.width[i])/2.0)**2*float(df_submit2.height[i])
      for i in df_submit2[df_submit2['shape'] == 'box'].index:
              df_submit2.volume[i] = float(df_submit2.width[i])*float(df_submit2.height[i])*float(df_submit2.depth[i])
      for i in df_submit2[df_submit2['shape'] == 'ellipsoid'].index:
              df_submit2.volume[i] = float(df_submit2.width[i])*float(df_submit2.height[i])*float(df_submit2.depth[i])*(math.pi/6.0)
      return df_submit2

def area(df_submit2):
      for i in df_submit2[df_submit2['shape'] == 'cylinder'].index:
              df_submit2.base_area[i] = math.pi*float(df_submit2.width[i]/2.0)**2
      for i in df_submit2[df_submit2['shape'] == 'box'].index:
              df_submit2.base_area[i] = float(df_submit2.width[i])*float(df_submit2.depth[i])
      for i in df_submit2[df_submit['shape'] == 'ellipsoid'].index:
              df_submit2.base_area[i] = float(df_submit2.width[i])*float(df_submit2.depth[i])*(math.pi/6.0)
      return df_submit2

df_submit2 = volume(df_submit2)
df_submit2 = area(df_submit2)
df_submit2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyW

,id,height,width,depth,shape,quality,volume,base_area
0,10001,67,26,26,ellipsoid,94,23714.835744,353.952772
1,10002,8,99,86,cylinder,50,61581.499196,7697.687399
2,10003,75,55,5,ellipsoid,65,10799.224747,143.989663
3,10004,32,47,52,box,1,78208.000000,2444.000000
4,10005,83,19,56,ellipsoid,34,46240.055071,557.109097
...,...,...,...,...,...,...,...,...
1995,11996,73,85,28,ellipsoid,3,90970.051272,1246.165086
1996,11997,36,32,54,ellipsoid,7,32572.032632,904.778684
1997,11998,36,47,48,ellipsoid,28,42524.598159,1181.238838
1998,11999,14,39,2,cylinder,23,16724.268491,1194.590607


In [29]:
# Feature engineering

# We should apply feature engineering to the variable shape

dummies = pd.get_dummies(df_train['shape'],prefix = "shape")
df_train = pd.concat([df_train,dummies],axis=1)

In [30]:
df_train.drop('shape', axis=1, inplace=True)


In [31]:
dummies = pd.get_dummies(df_submit['shape'],prefix = "shape")
df_submit2 = pd.concat([df_submit,dummies],axis=1)
df_submit2.drop('shape', axis=1, inplace=True)
df_submit2.drop('id', axis=1, inplace=True)


In [32]:
#df_train['cost'] = stats.zscore(df_train['cost'])
df_submit2

,height,width,depth,quality,volume,base_area,shape_box,shape_cylinder,shape_ellipsoid
0,67,26,26,94,23714.835744,353.952772,0,0,1
1,8,99,86,50,61581.499196,7697.687399,0,1,0
2,75,55,5,65,10799.224747,143.989663,0,0,1
3,32,47,52,1,78208.000000,2444.000000,1,0,0
4,83,19,56,34,46240.055071,557.109097,0,0,1
...,...,...,...,...,...,...,...,...,...
1995,73,85,28,3,90970.051272,1246.165086,0,0,1
1996,36,32,54,7,32572.032632,904.778684,0,0,1
1997,36,47,48,28,42524.598159,1181.238838,0,0,1
1998,14,39,2,23,16724.268491,1194.590607,0,1,0


In [33]:
df_submit2

,height,width,depth,quality,volume,base_area,shape_box,shape_cylinder,shape_ellipsoid
0,67,26,26,94,23714.835744,353.952772,0,0,1
1,8,99,86,50,61581.499196,7697.687399,0,1,0
2,75,55,5,65,10799.224747,143.989663,0,0,1
3,32,47,52,1,78208.000000,2444.000000,1,0,0
4,83,19,56,34,46240.055071,557.109097,0,0,1
...,...,...,...,...,...,...,...,...,...
1995,73,85,28,3,90970.051272,1246.165086,0,0,1
1996,36,32,54,7,32572.032632,904.778684,0,0,1
1997,36,47,48,28,42524.598159,1181.238838,0,0,1
1998,14,39,2,23,16724.268491,1194.590607,0,1,0


In [34]:
# Neural network

# Train a model to predict cost when given the following inputs: 
# 'height', 'width', 'depth', 'quality','shape_box','shape_cylinder','shape_ellipsoid'.

y = df_train.cost.values
x = df_train.drop(df_train.columns.difference(['height', 'width', 'depth', 'quality','shape_box','shape_cylinder','shape_ellipsoid','volume','base_area']), axis = 1).values
y.shape

# Neural network predicting cost - regularization methods are applied

from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers


# Cross-Validate + early stopping
kf = KFold(5, shuffle=True, random_state=42) # Use for KFold classification
    
oos_y = []
oos_pred = []

fold = 0
for train, test in kf.split(x):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    #  I added these layers just to try - I don't know if this approach is correct. Previously, I was using two layers.

    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation='relu',kernel_regularizer=regularizers.l2(1e-4), bias_regularizer=regularizers.l2(1e-4)))
    model.add(Dense(100, input_dim=x.shape[1], activation='relu',kernel_regularizer=regularizers.l2(1e-4), bias_regularizer=regularizers.l2(1e-4)))
    model.add(Dense(100, input_dim=x.shape[1], activation='relu',kernel_regularizer=regularizers.l2(1e-4), bias_regularizer=regularizers.l2(1e-4)))
    model.add(Dense(100, input_dim=x.shape[1], activation='relu',kernel_regularizer=regularizers.l2(1e-4), bias_regularizer=regularizers.l2(1e-4)))
    model.add(Dense(25, input_dim=x.shape[1], activation='relu',kernel_regularizer=regularizers.l2(1e-4), bias_regularizer=regularizers.l2(1e-4)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    custom_early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001)    
    model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=0,epochs=1500,callbacks=[custom_early_stopping])
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)    

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Fold score (RMSE): {score}")

# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final, out of sample score (RMSE): {score}")    
    
# Write the cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [oos_y, oos_pred],axis=1 )






Fold #1
Fold score (RMSE): 579.9972899298914
Fold #2
Fold score (RMSE): 724.3752828584459
Fold #3
Fold score (RMSE): 634.0994225604047
Fold #4
Fold score (RMSE): 663.9878691173261
Fold #5
Fold score (RMSE): 630.368730617129
Final, out of sample score (RMSE): 648.2967085075364


In [35]:
x

array([[40., 53., 89., ...,  0.,  1.,  0.],
       [85., 35., 51., ...,  1.,  0.,  0.],
       [12., 19., 61., ...,  1.,  0.,  0.],
       ...,
       [20., 69., 32., ...,  1.,  0.,  0.],
       [52., 95., 99., ...,  1.,  0.,  0.],
       [20., 82., 47., ...,  1.,  0.,  0.]])

In [36]:
# Prediction cost (this is 'submit' dataset)
# 'height', 'width', 'depth', 'quality','shape_box','shape_cylinder','shape_ellipsoid','volume,'base_area'.

x_submit = df_submit2.values
y_submit = model.predict(x_submit)
y_submit


array([[119.20767 ],
       [318.95105 ],
       [ 54.17688 ],
       ...,
       [220.79472 ],
       [ 84.54677 ],
       [ 24.198406]], dtype=float32)

In [37]:
df_submit2

,height,width,depth,quality,volume,base_area,shape_box,shape_cylinder,shape_ellipsoid
0,67,26,26,94,23714.835744,353.952772,0,0,1
1,8,99,86,50,61581.499196,7697.687399,0,1,0
2,75,55,5,65,10799.224747,143.989663,0,0,1
3,32,47,52,1,78208.000000,2444.000000,1,0,0
4,83,19,56,34,46240.055071,557.109097,0,0,1
...,...,...,...,...,...,...,...,...,...
1995,73,85,28,3,90970.051272,1246.165086,0,0,1
1996,36,32,54,7,32572.032632,904.778684,0,0,1
1997,36,47,48,28,42524.598159,1181.238838,0,0,1
1998,14,39,2,23,16724.268491,1194.590607,0,1,0


In [38]:
#y_submit = np.exp(y_submit)-1
#y_submit = df_train2['cost'].std()*y_submit+df_train2['cost'].mean()

In [39]:

y_submit

array([[119.20767 ],
       [318.95105 ],
       [ 54.17688 ],
       ...,
       [220.79472 ],
       [ 84.54677 ],
       [ 24.198406]], dtype=float32)

In [40]:
# Creating submit dataframe
#y_submit = (y_submit)*df_train['cost'].std()+df_train['cost'].mean()
y_submit = pd.DataFrame(y_submit)
df_submit = pd.concat([df_submit,y_submit],axis=1)
df_submit.rename(columns = {0:'cost'}, inplace = True) 
df_submit.drop(['height', 'width', 'depth', 'shape', 'quality','volume','base_area'], axis=1, inplace=True)
df_submit


,id,cost
0,10001,119.207672
1,10002,318.951050
2,10003,54.176880
3,10004,405.375244
4,10005,242.059555
...,...,...
1995,11996,473.428467
1996,11997,171.928619
1997,11998,220.794724
1998,11999,84.546768


In [41]:
#df_submit['cost'] = df_submit['cost']*df_train2['cost'].std()+df_train2['cost'].mean()

for i in range(len(df_submit.cost)):
    print(df_submit.cost[i])

119.20767
318.95105
54.17688
405.37524
242.05956
250.07439
587.9855
2636.3552
316.22656
370.45966
271.55154
1988.693
213.47003
732.06946
354.46173
453.22906
205.06494
21.307793
633.2201
129.12799
3024.6077
273.3747
2.639803
1416.838
37.087505
875.7533
23.077105
816.6141
5.119822
202.16148
371.4635
651.09375
334.41397
1073.1329
405.09586
973.7948
1764.4955
384.47882
543.00195
369.48267
149.76488
466.81006
938.9148
35.72274
900.0835
856.7222
30.476732
200.35783
1766.2533
737.7855
1.7684404
61.622066
131.87271
688.44385
87.91551
122.03189
343.3887
18.250051
354.93243
289.3448
2267.2341
932.67114
643.376
2957.216
221.1108
555.2803
1198.3007
341.8963
365.87296
375.5838
288.72797
354.72977
1097.5594
17.468887
2.2358847
138.02913
1223.579
264.6538
834.0609
15.409286
499.46558
362.4112
655.4705
1109.5326
0.31117976
22.397467
24.332964
546.7235
47.231075
1663.065
2892.4285
2286.852
980.58417
37.992985
33.193764
43.98174
18.727522
993.29285
271.54782
58.64412
689.87463
473.81287
315.1455
280.906

In [42]:
#submit function

submit(source_file=file,data=[df_submit],key=key,no=8)

Success: Submitted Assignment 8 for victor.macia:
You have submitted this assignment 160 times. (this is fine)
Note: The mean difference 0.9022847207050972 for column 'cost' is acceptable and is less than the maximum allowed value of '1.0' for this assignment.
No warnings or errors (only notes), you will probably do well, but no guarantee. :-)
